# Инференс модели в kserve

In [15]:
import json
import requests

Функция для отправки запроса к модели

In [13]:
def infer(text):
    api = 'http://kserve:8080/v1/models/NERtagger:predict'
    headers = {'Content-type': 'application/json', 'Accept': '*/*'}
    payload = json.dumps({"text": text})
    response = requests.post(api, data=payload, headers=headers)
    try:
        data = response.json()     
        return data                
    except requests.exceptions.RequestException:
        print(response.text)

Ответ модели отправляется в формате JSON:

```json
{'text': sentence_text,
 'entities': [
             {'text': entitie_text, 'type': entitie_type, 'start': start, 'end': end},
             ...
             {'text': entitie_text, 'type': entitie_type, 'start': start, 'end': end}
             ]
}
```

Здесь:

- *sentence_text* : текст предложения (запроса)
- *entities* : список сущностей
- *entitie_text* : текст сущности
- *entitie_type* : тип сущности
- *start* : начало сущности в исходном тексте
- *stop* : конец сущности в исходном тексте

Проверим ответ на предложении:

In [17]:
text='Глава компании Apple Тим Кук осенью представил новый телефон в штаб квартире Apple Inc в Купертино'

In [64]:
%time outputs = infer(text)
outputs

CPU times: user 3.78 ms, sys: 1.69 ms, total: 5.47 ms
Wall time: 518 ms


{'text': 'Глава компании Apple Тим Кук осенью представил новый телефон в штаб квартире Apple Inc в Купертино',
 'entities': [{'text': 'глава', 'type': 'O', 'start': 0, 'end': 5},
  {'text': 'компании', 'type': 'O', 'start': 0, 'end': 14},
  {'text': 'apple', 'type': 'ORG', 'start': 6, 'end': 20},
  {'text': 'тим кук', 'type': 'PER', 'start': 18, 'end': 28},
  {'text': 'осенью', 'type': 'O', 'start': 27, 'end': 35},
  {'text': 'представил', 'type': 'O', 'start': 29, 'end': 46},
  {'text': 'новыи', 'type': 'O', 'start': 36, 'end': 52},
  {'text': 'телефон', 'type': 'O', 'start': 51, 'end': 60},
  {'text': 'в', 'type': 'O', 'start': 53, 'end': 62},
  {'text': 'штаб', 'type': 'O', 'start': 61, 'end': 67},
  {'text': 'квартире', 'type': 'O', 'start': 63, 'end': 76},
  {'text': 'apple inc', 'type': 'ORG', 'start': 68, 'end': 86},
  {'text': 'в', 'type': 'O', 'start': 83, 'end': 88},
  {'text': 'купертино', 'type': 'LOC', 'start': 87, 'end': 98}]}

Проверим работу сервиса при множественных запросах

In [66]:
%time [infer(text) for i in range(100)]
None

CPU times: user 204 ms, sys: 37.1 ms, total: 241 ms
Wall time: 42.3 s


На 100 запросов понадобилось 42.3 секунды. В среднем 423 мс